<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prelude" data-toc-modified-id="Prelude-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prelude</a></span></li><li><span><a href="#Utilities-(as-before)" data-toc-modified-id="Utilities-(as-before)-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Utilities (as before)</a></span></li><li><span><a href="#Finding-Shortest-Path-(as-before)" data-toc-modified-id="Finding-Shortest-Path-(as-before)-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Finding Shortest Path (as before)</a></span></li><li><span><a href="#Finding-Shortest-Delivery-Route-(as-before)" data-toc-modified-id="Finding-Shortest-Delivery-Route-(as-before)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Finding Shortest Delivery Route (as before)</a></span></li><li><span><a href="#Time-Handling" data-toc-modified-id="Time-Handling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Time Handling</a></span></li><li><span><a href="#Class-Recorder" data-toc-modified-id="Class-Recorder-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Class Recorder</a></span></li><li><span><a href="#Class-Parcel" data-toc-modified-id="Class-Parcel-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Class Parcel</a></span></li><li><span><a href="#Class-Customer" data-toc-modified-id="Class-Customer-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Class Customer</a></span></li><li><span><a href="#Class-Driver" data-toc-modified-id="Class-Driver-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Class Driver</a></span></li><li><span><a href="#Class-Delivery-Centre" data-toc-modified-id="Class-Delivery-Centre-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Class Delivery Centre</a></span></li><li><span><a href="#Simulation" data-toc-modified-id="Simulation-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Simulation</a></span><ul class="toc-item"><li><span><a href="#Parameters-from-Specification" data-toc-modified-id="Parameters-from-Specification-11.1"><span class="toc-item-num">11.1&nbsp;&nbsp;</span>Parameters from Specification</a></span></li><li><span><a href="#Generate-Input-Data" data-toc-modified-id="Generate-Input-Data-11.2"><span class="toc-item-num">11.2&nbsp;&nbsp;</span>Generate Input Data</a></span></li><li><span><a href="#Simulation-Routine" data-toc-modified-id="Simulation-Routine-11.3"><span class="toc-item-num">11.3&nbsp;&nbsp;</span>Simulation Routine</a></span></li><li><span><a href="#Model-Verification" data-toc-modified-id="Model-Verification-11.4"><span class="toc-item-num">11.4&nbsp;&nbsp;</span>Model Verification</a></span></li></ul></li></ul></div>

# Prelude

In [1]:
import matplotlib.pyplot as plt
import pulp
import math
import random
import pandas as pd
import numpy as np
import simpy

# Utilities (as before)

## Points and Distances

In [2]:
def dist(p1, p2):
    (x1, y1) = p1
    (x2, y2) = p2
    return int(math.sqrt((x1-x2)**2+(y1-y2)**2))

## PlotMap

In [3]:
def plotMap(G, T=[], P=[], W=None,
            style='r-o', lw=1, ms=3, 
            styleT='go', msT=5,
            styleP='b-o', lwP=3, msP=1,
            stylePT='go', msPT=7,
            styleW='bo', msW=9,
            text=None, grid=False):
    fig = plt.gcf()
    fig.set_size_inches(6, 6)
    V, E = G

    if not grid:
        plt.axis('off')
    plt.plot( [ p[0] for p in V ], [ p[1] for p in V ], 'ro', lw=lw, ms=ms)
    for (p, q) in E:
        plt.plot( [ p[0], q[0] ], [ p[1], q[1] ], 'r-o', lw=lw, ms=ms)
    for t in T:
        plt.plot( [ t[0] ], [ t[1] ], 
                  styleT, ms=msT)
    plt.plot( [ p[0] for p in P ],
              [ p[1] for p in P ], 
              styleP, lw=lwP, ms=msP)
    for p in P:
        if p in T:
            plt.plot( [ p[0] ], [ p[1] ], 
                      stylePT, ms=msPT)
    if W is not None:
        plt.plot( [ W[0] ], [ W[1] ], 
                      styleW, ms=msW)
    if text is not None:
        maxX = max([p[0] for p in V])
        plt.text(0.8*maxX, 0, text)
    if grid:
        plt.grid()
    plt.show()

## Add Targets

In [4]:
def addTargets(M, T):
    V, E = M
    E = E.copy()
    V = V.copy()
    for t in T:
        minD = math.inf
        minE = None
        for e in E:
            P, Q = e
            distT = dist(P, t)+dist(t, Q)-dist(P, Q)
            if distT < minD:
                minD = distT
                minE = e
        P, Q = minE
        E.remove( (P, Q) )
        E.append( (P, t) )
        E.append( (t, Q) )
        V.append(t)
    return V, E

## Generate Warehouse Location

In [5]:
def generateWarehouseLocation(M):
    V, _ = M
    W = random.sample(V, k=1)[0]
    return W

## Time Handling

**Convention:** In this project we measure time in seconds. The simulation will start at 0:00. Time related methods will be added as they are needed.

`timestamp(t)` generates a timestamp string in the form `[dd] hh:mm:ss.d`

In [6]:
    def timestamp(t):
        day = int(t)//(24*3600)
        t = t - day*24*3600
        hour = int(t)//3600
        t = t - hour*3600
        mins = int(t)//60
        t = t - mins*60
        secs = int(t)
        t = int(round((t - secs)*10))
        return f"[{day:2d}] {hour:02d}:{mins:02d}:{secs:02d}.{t:1d}"

In [7]:
timestamp(24*3600*3+17*3600+615.1)

'[ 3] 17:10:15.1'

In [8]:
def nextHour(env, hour):
    beginningOfDay = int(env.now//(24*3600))*24*3600
    timeOfDay = env.now-beginningOfDay
    if hour*3600 > timeOfDay:
        return hour*3600 - timeOfDay
    else:
        return hour*3600 + 24*3600 - timeOfDay

In [9]:
def day(now):
    return int(now//(24*3600))

# Finding Shortest Path (as before)

In [10]:
def dist(p1, p2):
    (x1, y1) = p1
    (x2, y2) = p2
    return int(math.sqrt((x1-x2)**2+(y1-y2)**2))

In [11]:
def pathLength(P):
    return 0 if len(P)<=1 else \
            dist(P[0], P[1])+pathLength(P[1:])

In [12]:
def shortestPath(M, A, B):

    def h(p): 
        return pathLength(p)+dist(p[-1],B)
    
    # candidates C are pairs of the path so far and 
    # the heuristic function of that path, 
    # sorted by the heuristic function, as maintained by
    # insert function
    def insert(C, p):
        hp = h(p)
        c = (p, hp)
        for i in range(len(C)):
            if C[i][1]>hp:
                return C[:i]+[c]+C[i:]
        return C+[c]   
        
    V, E = M
    assert(A in V and B in V)    
    C = insert([], [A])

    while len(C)>0:
        # take the first candidate out of the list of candidates
        path, _ = C[0]
        C = C[1:]
        if path[-1]==B:
            return path
        else:
            for (x, y) in E:
                if path[-1]==x and y not in path:
                    C = insert(C, path+[y])
                elif path[-1]==y and x not in path:
                    C = insert(C, path+[x])
    return None

# Finding Shortest Delivery Route (as before)

## Iterative Integer Programming

In [13]:
def createTables(M, T):

    def reverse(P):
        return [ P[-i] for i in range(1,len(P)+1) ]
    
    def index(x, L):
        for i in range(len(L)):
            if x==L[i]: 
                return i
        return None

    n = len(T)
    d = [ [ math.inf for t in T ] for t in T ]
    p = [ [ None for t in T ] for t in T ]
    for i in range(n):
        d[i][i] = 0
        p[i][i] = [ T[i] ]
    for i in range(n):
        for j in range(n):
            if p[i][j] is None:
                s = shortestPath(M, T[i], T[j])
                d[i][j] = d[j][i] = pathLength(s)
                p[i][j] = s
                p[j][i] = reverse(s)
                for m in range(len(s)-1):
                    smi = index(s[m], T)
                    if smi is None:
                        continue
                    for l in range(m+1, len(s)):
                        sli = index(s[l], T)
                        if sli is None:
                            continue  
                        sub = s[m:l+1]
                        if p[smi][sli] is None:
                            p[smi][sli] = sub
                            p[sli][smi] = reverse(sub)
                            d[smi][sli] = d[sli][smi] = pathLength(sub)
    return d,p               

In [14]:
def roundtrips(x, n):
    
    def isElem(x, l):
        for i in range(len(l)):
            if l[i]==x:
                return True
        return False

    def startpoint(trips):
        for i in range(n):
            for t in trips:
                if isElem(i, t):
                    break
            else:
                return i
    
    def totalLength(trips):
        s=0
        for i in range(0, len(trips)):
            s += len(trips[i])-1
        return s

    trips = []
    while totalLength(trips)<n:
        start = startpoint(trips)
        trip = [ start ]
        i = start
        while len(trip) < n-totalLength(trips):
            for j in range(0, n):
                if pulp.value(x[i][j])==1:
                    trip.append(j)
                    i=j
                    break        
            if pulp.value(x[trip[-1]][start])==1:
                trip.append(start)
                break
        trips.append(trip)
    return sorted(trips, key=lambda t: len(t), reverse=True)

In [15]:
import time

def createLoop(M, T, timing=False):
    
    if timing:
        start_time = time.time()
        last_time = time.time()
        
    D, P = createTables(M, T)   # These are the distances between customers and warehouse only
    
    if timing:
        print(f"createTables:   {time.time()-start_time:6.2f}s")
        last_time = time.time()
        
    n = len(T)

    if n==1:
        return T
        
    # create variables
    x = pulp.LpVariable.dicts("x", ( range(n), range(n) ),
                            lowBound=0, upBound=1, cat=pulp.LpInteger)
    # create problem
    prob = pulp.LpProblem("Loop",pulp.LpMinimize)
    # add objective function
    prob += pulp.lpSum([ D[i][j]*x[i][j] 
                             for i in range(n) for j in range(n) ])
    # add constraints
    constraints=0
    for j in range(n):
        prob += pulp.lpSum([ x[i][j] for i in range(n) if i!=j ]) ==1
    constraints += n
    for i in range(n):
        prob += pulp.lpSum([ x[i][j] for j in range(n) if i!=j ]) ==1
    constraints += n
    for i in range(n):
        for j in range(n):
            if i!=j:
                prob += x[i][j]+x[j][i] <= 1
                constraints += 1
    # initialise solver
    solvers = pulp.listSolvers(onlyAvailable=True)
    solver = pulp.getSolver(solvers[0], msg=0)
    prob.solve(solver)

    if timing:
        print(f"Solver:         {time.time()-last_time:6.2f}s {constraints:6,d} Constraints")
        last_time = time.time()

    trips = roundtrips(x, n)
    while len(trips)>1:
        longest = max([ len(t) for t in trips ])
        for t in trips:
            if len(t)<longest:
                prob += pulp.lpSum([ x[t[i]][t[i+1]] + x[t[i+1]][t[i]]
                                        for i in range(0,len(t)-1) ]) <= len(t)-2
                constraints += 1
            else:
                longest = math.inf
        prob.solve(solver)
        
        if timing:
            print(f"Solver:         {time.time()-last_time:6.2f}s {constraints:6,d} Constraints")
            last_time = time.time()

        trips = roundtrips(x, n)
        
    trip = trips[0]
    loop = []
    for k in range(len(trip)-1):
        sub = P[trip[k]][trip[k+1]]
        loop += sub if len(loop)==0 else sub[1:]
        
    if timing:
        print(f"createLoop:     {time.time()-start_time:6.2f}s")

    return loop

## Heuristic Algorithm

In [16]:
def FW(M):
    
    V, E = M

    n = len(V)
    d = [ [ math.inf for j in range(n) ] for i in range(n) ]
    p = [ [ None for j in range(n) ] for i in range(n) ]

    for (A, B) in E:
        a = V.index(A)
        b = V.index(B)
        d[a][b] = d[b][a] = dist(A, B)
        p[a][b] = [A, B]
        p[b][a] = [B, A]
    
    for i in range(n):
        d[i][i] = 0
        p[i][i] = [V[i]]
    
    for k in range(n):
        for i in range(n):
            for j in range(n):
                dk = d[i][k] + d[k][j]
                if d[i][j] > dk:
                    d[i][j] = dk
                    p[i][j] = p[i][k][:-1] + p[k][j]
                    
    return d, p

In [17]:
def createLoopH(M, T, timing=False):
    
    def makeLoop(L):
        loop = []
        for i in range(len(L)-1):
            A = L[i]
            B = L[i+1]
            a = V.index(A)
            b = V.index(B)
            sub = P[a][b]
            loop += sub if len(loop)==0 else sub[1:]
        return loop
        
    if timing:
        start_time = time.time()
        last_time = time.time()
        
    V, E = M
    D, P = FW(M)   # note these are the distances between all vertices in M (and T)
    
    if timing:
        print(f"createTables:   {time.time()-start_time:6.2f}s")
        last_time = time.time()
        
    W = T[0]
    customers = T[1:]
    if len(T)==1:
        L = T
    elif len(T)<=3:
        L = T + [T[0]]
    else:
        L = T[:3]+[T[0]]
        T = T[3:]
        while len(T)>0:
            minExt = math.inf
            minInd = None
            selInd = None
            for k in range(len(T)):
                C = T[k]
                c = V.index(C)
                for i in range(0, len(L)-1):
                    A = L[i]
                    B = L[i+1]
                    a = V.index(A)
                    b = V.index(B)
                    ext = D[a][c] + D[c][b] - D[a][b]
                    if ext<minExt:
                        minExt, minInd, selInd = ext, i+1, k
            L = L[:minInd]+[T[selInd]]+L[minInd:]
            T = T[:selInd]+T[selInd+1:]
            
    if timing:
        print(f"createLoopH:    {time.time()-start_time:6.2f}s")

    return makeLoop(L)

# Class Recorder

We willl use a class Recorder as a reference point for capturing data during the simulation. There will be only one recorder. It will be created at the beginning of every simulation run. Every entity will carry a reference to the Recorder. 

In [18]:
import time

class Recorder:
    
    def __init__(self, env, M, W, C, days, 
                 log=False, plot=False, timing=False):
        self.env = env
        self.M = M
        self.W = W
        self.C = C
        self.days = days
        self.log = log
        self.plot = plot

        self.timing = timing
        self.start_time = time.time()
        self.last_time = self.start_time
        self.cum_timer = {}
            
        Customer.REGISTER = []
        Parcel.REGISTER = []
    
    def timer(self, s):
        t = time.time()
        Δt = t-self.last_time
        if self.timing:
            print(f"==== t: {t-self.start_time:6.2f}s "
                  f"Δt: {Δt:6.2f}s [{s:s}]")
        if s in self.cum_timer:
            self.cum_timer[s] += Δt
        else:
            self.cum_timer[s] = Δt
        self.last_time = t

    def reportTimer(self):
        print()
        for k in sorted(self.cum_timer, key=lambda x: self.cum_timer[x], reverse=True):
            print(f"==== Σ: {self.cum_timer[k]:6.2f}s [{k:s}]")
        print(f"==== -----------------------")
        print(f"==== Σ: {time.time()-self.start_time:6.2f}s Total")
 
    def trace(self, event):
        if self.log:
            print(timestamp(self.env.now), event)

    def finish(self):
        # simulation is finished for good
        # by removing the simulation environment we can
        # pickle recorder
        self.env = None

# Class Parcel

<img src='img/EventGraph Parcel.png' width=500/>

Parcels follow through a sequence of states:
- processing
- in transit (from manufacture to distribution centre)
- arrived in distribution centre
- ready for delivery 
- out for delivery
- customer not present
- returned to distribution centre
- delivered

In [19]:
class Parcel:
    
    REGISTER = []
    
    def __init__(self, rec, i, cust, custIndex):
        self.rec = rec
        self.i = i  # row index in data frames of input data
        self.dest = cust.location
        self.custIndex = custIndex
        self.status = [ 'processing' ] # status record and
        self.timing = [ self.rec.env.now ]      # timing
        assert(len(Parcel.REGISTER)==i)
        Parcel.REGISTER += [ self ] 
    
    # factory method ensures that there is only
    # one Parcel per location
    def getParcel(rec, i, location, custIndex):
        for p in Parcel.REGISTER:
            if p.i == i:
                return p
        return Parcel(rec, i, location, custIndex)

    
    def __str__(self):
        return f"Parcel: {self.i:3d} ({self.custIndex:3d})"

    def index(self):
        return self.i
    
    def destination(self):
        return self.dest
    
    def __reg(self, state):
        self.status += [ state ]
        self.timing += [ self.rec.env.now ]
        self.rec.trace(str(self)+" "+state)
        
    def arrivedAtDeliveryCentre(self):
        self.__reg('arr at delivery centre')
        
    def outForDelivery(self): 
        self.__reg('out for delivery')
        
    def returnFromDelivery(self):
        self.__reg('return from delivery')
    

# Class Customer

<img src='img/EventGraph Customer.png' width=500/>

In [20]:
class Customer:
    
    REGISTER = []

    def __init__(self, rec, location):
        self.rec = rec
        self.location = location
        self.i = len(Customer.REGISTER)
        Customer.REGISTER += [ self ]
        self.atHome = True
        self.answersDoor = False
        self.parcelsReceived = []
        rec.env.process(self.process())
        
    def __str__(self):
        return f"Customer: {self.i:2d} {str(self.location):s}"
    
    # factory method ensures that there is only
    # one customer per location
    def getCustomer(rec, location):
        for c in Customer.REGISTER:
            if c.location == location:
                return c
        return Customer(rec, location)
    
    def leaveHouse(self):
        assert(self.atHome and not self.answersDoor)
        # self.rec.trace(str(self)+" leaves house")
        self.atHome = False
     
    def returnHome(self):
        assert(not self.atHome)
        # self.rec.trace(str(self)+" returns home")
        self.atHome = True
        
    def answerDoor(self):
        if self.atHome:
            yield self.rec.env.timeout(random.expovariate(1/AVERAGE_TIME_ANSWER_DOOR))
            self.rec.trace(str(self)+" answers door")
            self.answersDoor = True
        else:
            yield self.rec.env.timeout(WAIT_TIME_IF_CUSTOMER_DOESNT_ANSWER_DOOR)
            self.rec(str(self)+" not at home")
            
    def acceptParcel(self, parcel):
        assert(self.answersDoor)
        self.parcelsReceived += [parcel]
        self.rec.trace(str(self)+" accepts "+str(parcel))
        
    def signOff(self):
        assert(self.answersDoor)
        self.rec.trace(str(self)+" signs off")
        self.answersDoor = False
        
    def process(self):
        yield self.rec.env.timeout(nextHour(self.rec.env, 8))
        while day(self.rec.env.now)<self.rec.days:
            # in a refinement we may use random times
            self.leaveHouse()
            yield self.rec.env.timeout(nextHour(self.rec.env, 18))
            self.returnHome()
            yield self.rec.env.timeout(nextHour(self.rec.env, 8))

# Class Driver

<img src='img/EventGraph Driver.png' width=500/>

In [21]:
class Driver:
    
    def __init__(self, rec, DC):
        self.rec = rec
        self.DC = DC
        self.location = None
        self.parcels = None
        self.tour = None
        self.rec.env.process(self.process())
        
    # activity
    def __drive(self, target):
        assert(self.tour[0] == self.location)
        while self.location!=target:
            d = dist(self.location, self.tour[1])
            yield self.rec.env.timeout(d / AVERAGE_SPEED)
            self.location = self.tour[1]
            self.tour = self.tour[1:]
        assert(self.tour[0] == self.location == target)
    
    def arriveForWork(self):
        self.location = self.DC.W
        self.parcels = []
        self.returns = []
        self.tour = [ self.DC.W ]
        self.rec.trace("Driver arrives for work")
        
    def leaveForDelivery(self, tour, parcels):
        self.tour, self.parcels = tour, parcels
        self.rec.trace(f"Driver leaves for delivery " \
                       f"of {len(parcels):d} parcels")
        
    def process(self):
        yield self.rec.env.timeout(nextHour(self.rec.env, 18))
        while day(self.rec.env.now)<self.rec.days:
            self.arriveForWork()
            tour, parcels = self.DC.sendForDelivery()
            yield self.rec.env.timeout(PREP_TIME_PER_PARCEL*len(parcels))
            self.leaveForDelivery(tour, parcels)        
            while len(self.parcels)>0:
                # drive to customer
                custLocation = self.parcels[0].dest
                cust = Customer.getCustomer(self.rec, custLocation)
                self.rec.trace("Driver drives to "+str(cust))
                yield from self.__drive(custLocation)
                self.rec.trace("Driver arrived at "+str(cust))
                # call at customer
                yield from cust.answerDoor()
                
                if cust.answersDoor:
                    while len(self.parcels)>0 and \
                            custLocation == self.parcels[0].dest:
                        cust.acceptParcel(self.parcels[0])
                        yield self.rec.env.timeout(random.expovariate(1/10))
                        self.parcels = self.parcels[1:]
                    cust.signOff()
                    yield self.rec.env.timeout(random.expovariate(1/10))
                else:
                    while len(self.parcels)>0 and  \
                            custLocation == self.parcels[0].dest:
                        self.returns += self.parcels[0]
                        self.parcels = self.parcels[1:]

            # return to delivery centre
            self.rec.trace("Driver returns to delivery centre")
            yield from self.__drive(self.DC.W)
            self.rec.trace("Driver arrived at delivery centre")
            
            for parcel in self.returns:
                self.DC.returnFromDelivery(parcel)
                yield self.rec.env.timeout(RETURN_TIME_PER_PARCEL)
            yield self.rec.env.timeout(600)
        
            self.rec.trace("Driver goes home")
            
            yield self.rec.env.timeout(nextHour(self.rec.env, 18))

# Class Delivery Centre

<img src="img/EventGraph Delivery Centre.png" width=300/>

In [22]:
class DeliveryCentre:
    
    def __init__(self, rec, M, W):
        self.rec = rec
        self.M = M
        self.W = W
        self.limit = 35000
        
        self.leftOver = []    # list of parcels
        self.parcels = []     # list of parcels scheduled for delivery
        self.dest = []        # list of unique customer destinations
        self.tour = None      # tour planned for delivery
    
    def __accept(self, parcel):
        custLoc = parcel.dest
        if custLoc not in self.dest:
            MT = addTargets(self.M, self.dest + [custLoc])
            self.rec.timer("addTarget")
            SH = createLoopH(MT, [self.W] + self.dest + [custLoc], 
                             timing=self.rec.timing)
            self.rec.timer("createLoopH")
            if self.tour is None and pathLength(SH)<self.limit:
                self.parcels.append(parcel)
                self.dest += [custLoc]
            else: 
                S = createLoop(MT, [self.W] + self.dest + [custLoc], 
                               timing=self.rec.timing)
                self.rec.timer("createLoop")
                if pathLength(S)<self.limit:
                    self.parcels.append(parcel)
                    self.dest += [custLoc]
                    self.tour = S
                else:
                    self.leftOver.append(parcel)
        else:
            self.parcels.append(parcel)
        
    def acceptParcel(self, parcel):
        parcel.arrivedAtDeliveryCentre()
        self.__accept(parcel)
            
    def sendForDelivery(self):
        parcels = []
        if self.tour is None:
            MT = addTargets(self.M, self.dest)
            self.rec.timer("addTarget")
            self.tour = createLoop(MT, [self.W] + self.dest, 
                                   timing=self.rec.timing)
            self.rec.timer("createLoop")
        tour = self.tour
        addresses = self.dest
        
        # pick parcels in sequence to be delivered
        for i in range(1, len(tour)-1):
            dest = tour[i]
            for p in self.parcels:
                if p.dest == dest and p not in parcels:
                    parcels += [p]
                    p.outForDelivery()
         
        # arrange the left overs
        L = self.leftOver
        self.tour = None
        self.parcels = []
        self.leftOver = []
        self.dest = []
        for p in L:
            self.__accept(p)
        
        if self.rec.plot:
            plotMap(self.rec.M, T=addresses, P=tour, W=tour[0], 
                    text=f"Day {day(self.rec.env.now):2d}, {pathLength(tour):,d}m")

        return tour, parcels
                 
    def returnFromDelivery(self, parcel):
        parcel.returnFromDelivery()
        self.__accept(parcel)

    def getInventory(self):
        return len(self.parcels)+len(self.leftOver)

# Simulation

## Parameters from Specification

The time required for driving is based on the distance between way points at an average speed of 15km/h.

In [23]:
AVERAGE_SPEED = 15/3.6

The **cumulative preparation time** (route planning and 
sorting of the parcels in the delivery order and 
packing the cargo-bike) is assumed to be 50 sec per 
parcel to be delivered.

In [24]:
PREP_TIME_PER_PARCEL = 50

**Additional assumption:** The time to **process returned parcels** in the delivery centre is 30 sec per parce. 

In [25]:
RETURN_TIME_PER_PARCEL = 30

The average time to answer the door.

In [26]:
AVERAGE_TIME_ANSWER_DOOR = 40

In [27]:
WAIT_TIME_IF_CUSTOMER_DOESNT_ANSWER_DOOR = 60

## Generate Input Data

In [28]:
def generateDeliveryData(p, C, days, seed=0):
    ## p is the average number of parcels per day per customer
    ## C is the number of customers to be served
    ## days is the number of days for which data are to be generated.
    np.random.seed(seed)
    R = np.random.poisson(lam=len(C)*p, size=days)
    D = [ sorted(list(np.random.choice(range(len(C)), size=i))) for i in R ]
    return D

In [29]:
def generateInputData(D, log=False):

    R = [ len(d) for d in D ]
    N = sum(R)
    
    DAY_LENGTH = 24*3600    # measured in minutes
    DAY_START = 8*3600      # first delivery in the morning
    DAY_END = 17*3600     # last delivery during day time

    x = pd.DataFrame()

    x['iarr'] = [None]*N
    x['time'] = [None]*N
    x['day'] = [None]*N
    x['dest'] = [None]*N

    current_day = 0
    last_time = 0
    i = 0

    for d in D: # for each day

        if log:
            print("generating for day: ",current_day, D[current_day])
            
        time = current_day*DAY_LENGTH + DAY_START 

        for c in d: # for each customer that should get a 

            IARR = (DAY_END-DAY_START-2*3600) / len(d)  # estimated average IAT for the current day

            iat = random.expovariate(1.0/IARR)
            new_time = time + iat

            x.at[i, 'iarr'] = round(new_time - last_time,1)
            x.at[i, 'time'] = round(new_time - current_day*DAY_LENGTH , 1)
            x.at[i, 'day'] = current_day
            x.at[i, 'dest'] = c
            
            i += 1
            last_time = time = new_time
    
        current_day += 1
    
    return x

## Simulation Routine

In [30]:
def simulation(M, W, C, p=0.2, days=10, seed=0, 
               log=False, plot=False, timing=False):

    random.seed(seed)
    D = generateDeliveryData(p, C, days, seed)
    X = generateInputData(D, log=log)
    
    env = simpy.Environment()
    rec = Recorder(env, M, W, C, days, 
                   log=log, plot=plot, timing=timing)

    print(f"Simulating delivery of {len(X):d} parcels "
          f"over {len(D):d} days to {len(C):d} customers")

    for c in C:
        Customer.getCustomer(rec, c)           
    DC = DeliveryCentre(rec, M, W)
    D = Driver(rec, DC)

    def generatorProcess(env):
        # generate the parcels based on input data x
        for i in range(len(X)):
            yield env.timeout(X.at[i, 'iarr'])
            custIndex = X.at[i, 'dest']
            custLoc = C[custIndex]
            cust = Customer.getCustomer(rec, custLoc)
            p = Parcel.getParcel(rec, i, cust, custIndex)
            DC.acceptParcel(p)

    env.process(generatorProcess(env))
    env.run()

    rec.finish()

    if log:
        print(f"Delivery Centre Inventory: {DC.getInventory():d} parcels")

    return rec

## Model Verification

In [31]:
import pickle
with open('data.pickled', 'rb') as f:
    M, C = pickle.load(f)

In [33]:
random.seed(6077)
W = generateWarehouseLocation(M)
rec = simulation(M, W, C, p=0.15, days=30,seed=6077,   log=True)
rec.reportTimer()

generating for day:  0 [3, 8, 18, 20, 27, 32, 34, 40, 45, 50, 52, 55, 69, 71, 73, 74, 79, 85, 86, 91, 94, 101, 122, 124, 135, 135, 145]
generating for day:  1 [1, 8, 10, 13, 34, 39, 46, 58, 67, 71, 78, 81, 100, 100, 108, 127, 127, 129, 134, 137, 140, 141, 143, 147]
generating for day:  2 [0, 3, 6, 15, 18, 25, 32, 34, 35, 42, 42, 47, 49, 52, 53, 61, 74, 77, 91, 96, 102, 114, 119, 121, 132, 142, 146]
generating for day:  3 [10, 14, 17, 29, 29, 31, 41, 58, 72, 85, 89, 96, 109, 110, 112, 124, 141]
generating for day:  4 [12, 19, 20, 51, 58, 75, 76, 86, 92, 98, 100, 107, 135, 137, 140, 146]
generating for day:  5 [8, 11, 12, 29, 30, 76, 78, 82, 84, 110, 117, 119, 125, 126, 126, 134, 145]
generating for day:  6 [2, 8, 14, 15, 25, 40, 55, 90, 97, 103, 112, 116, 120, 141, 149]
generating for day:  7 [4, 10, 15, 20, 30, 33, 37, 48, 61, 67, 68, 82, 82, 96, 98, 101, 101, 104, 108, 108, 116, 118, 127, 133, 136, 141]
generating for day:  8 [10, 14, 15, 17, 33, 34, 46, 50, 51, 56, 63, 77, 91, 92, 94

[ 1] 08:13:27.0 Parcel:  28 (  8) arr at delivery centre
[ 1] 08:19:44.8 Parcel:  29 ( 10) arr at delivery centre
[ 1] 08:43:16.5 Parcel:  30 ( 13) arr at delivery centre
[ 1] 08:45:44.6 Parcel:  31 ( 34) arr at delivery centre
[ 1] 09:15:03.3 Parcel:  32 ( 39) arr at delivery centre
[ 1] 09:35:09.7 Parcel:  33 ( 46) arr at delivery centre
[ 1] 09:42:31.8 Parcel:  34 ( 58) arr at delivery centre
[ 1] 10:00:52.1 Parcel:  35 ( 67) arr at delivery centre
[ 1] 10:05:38.8 Parcel:  36 ( 71) arr at delivery centre
[ 1] 10:06:31.2 Parcel:  37 ( 78) arr at delivery centre
[ 1] 10:15:32.6 Parcel:  38 ( 81) arr at delivery centre
[ 1] 10:20:51.2 Parcel:  39 (100) arr at delivery centre
[ 1] 10:53:36.7 Parcel:  40 (100) arr at delivery centre
[ 1] 10:55:10.9 Parcel:  41 (108) arr at delivery centre
[ 1] 11:24:54.2 Parcel:  42 (127) arr at delivery centre
[ 1] 11:25:48.9 Parcel:  43 (127) arr at delivery centre
[ 1] 11:33:30.1 Parcel:  44 (129) arr at delivery centre
[ 1] 12:31:27.1 Parcel:  45 (13

[ 2] 08:55:40.6 Parcel:  53 (  6) arr at delivery centre
[ 2] 09:01:35.9 Parcel:  54 ( 15) arr at delivery centre
[ 2] 09:21:40.9 Parcel:  55 ( 18) arr at delivery centre
[ 2] 09:27:47.4 Parcel:  56 ( 25) arr at delivery centre
[ 2] 09:29:06.6 Parcel:  57 ( 32) arr at delivery centre
[ 2] 09:43:06.8 Parcel:  58 ( 34) arr at delivery centre
[ 2] 10:00:26.5 Parcel:  59 ( 35) arr at delivery centre
[ 2] 10:30:01.5 Parcel:  60 ( 42) arr at delivery centre
[ 2] 10:33:20.0 Parcel:  61 ( 42) arr at delivery centre
[ 2] 10:44:47.9 Parcel:  62 ( 47) arr at delivery centre
[ 2] 10:55:49.7 Parcel:  63 ( 49) arr at delivery centre
[ 2] 11:49:50.8 Parcel:  64 ( 52) arr at delivery centre
[ 2] 11:51:41.8 Parcel:  65 ( 53) arr at delivery centre
[ 2] 11:51:59.1 Parcel:  66 ( 61) arr at delivery centre
[ 2] 12:16:07.0 Parcel:  67 ( 74) arr at delivery centre
[ 2] 12:36:33.8 Parcel:  68 ( 77) arr at delivery centre
[ 2] 12:44:32.9 Parcel:  69 ( 91) arr at delivery centre
[ 2] 12:45:15.0 Parcel:  70 ( 9

[ 3] 08:41:38.3 Parcel:  80 ( 17) arr at delivery centre
[ 3] 08:58:43.6 Parcel:  81 ( 29) arr at delivery centre
[ 3] 09:21:01.9 Parcel:  82 ( 29) arr at delivery centre
[ 3] 09:43:38.6 Parcel:  83 ( 31) arr at delivery centre
[ 3] 10:05:59.5 Parcel:  84 ( 41) arr at delivery centre
[ 3] 10:08:47.3 Parcel:  85 ( 58) arr at delivery centre
[ 3] 11:28:16.0 Parcel:  86 ( 72) arr at delivery centre
[ 3] 12:16:19.0 Parcel:  87 ( 85) arr at delivery centre
[ 3] 12:22:27.8 Parcel:  88 ( 89) arr at delivery centre
[ 3] 12:27:19.9 Parcel:  89 ( 96) arr at delivery centre
[ 3] 12:57:24.6 Parcel:  90 (109) arr at delivery centre
[ 3] 12:58:13.5 Parcel:  91 (110) arr at delivery centre
[ 3] 13:12:22.2 Parcel:  92 (112) arr at delivery centre
[ 3] 13:40:00.2 Parcel:  93 (124) arr at delivery centre
[ 3] 14:02:23.1 Parcel:  94 (141) arr at delivery centre
[ 3] 18:00:00.0 Driver arrives for work
[ 3] 18:00:00.0 Parcel:  88 ( 89) out for delivery
[ 3] 18:00:00.0 Parcel:  85 ( 58) out for delivery
[ 3

[ 4] 12:07:49.3 Parcel: 104 ( 98) arr at delivery centre
[ 4] 13:05:10.6 Parcel: 105 (100) arr at delivery centre
[ 4] 13:25:04.4 Parcel: 106 (107) arr at delivery centre
[ 4] 15:04:26.6 Parcel: 107 (135) arr at delivery centre
[ 4] 15:30:21.1 Parcel: 108 (137) arr at delivery centre
[ 4] 15:51:43.2 Parcel: 109 (140) arr at delivery centre
[ 4] 16:03:05.5 Parcel: 110 (146) arr at delivery centre
[ 4] 18:00:00.0 Driver arrives for work
[ 4] 18:00:00.0 Parcel:  99 ( 58) out for delivery
[ 4] 18:00:00.0 Parcel:  98 ( 51) out for delivery
[ 4] 18:00:00.0 Parcel:  95 ( 12) out for delivery
[ 4] 18:00:00.0 Parcel:  96 ( 19) out for delivery
[ 4] 18:00:00.0 Parcel:  97 ( 20) out for delivery
[ 4] 18:00:00.0 Parcel: 102 ( 86) out for delivery
[ 4] 18:00:00.0 Parcel: 103 ( 92) out for delivery
[ 4] 18:00:00.0 Parcel: 100 ( 75) out for delivery
[ 4] 18:00:00.0 Parcel: 101 ( 76) out for delivery
[ 4] 18:00:00.0 Parcel: 104 ( 98) out for delivery
[ 4] 18:00:00.0 Parcel: 110 (146) out for delivery


...

[24] 09:23:24.0 Parcel: 517 ( 18) arr at delivery centre
[24] 09:49:29.2 Parcel: 518 ( 25) arr at delivery centre
[24] 10:06:06.1 Parcel: 519 ( 26) arr at delivery centre
[24] 10:27:16.4 Parcel: 520 ( 39) arr at delivery centre
[24] 10:36:05.6 Parcel: 521 ( 41) arr at delivery centre
[24] 10:56:34.9 Parcel: 522 ( 66) arr at delivery centre
[24] 11:16:49.5 Parcel: 523 ( 67) arr at delivery centre
[24] 11:40:34.7 Parcel: 524 ( 85) arr at delivery centre
[24] 13:04:11.3 Parcel: 525 (114) arr at delivery centre
[24] 13:06:03.9 Parcel: 526 (119) arr at delivery centre
[24] 13:08:59.9 Parcel: 527 (120) arr at delivery centre
[24] 13:12:11.4 Parcel: 528 (122) arr at delivery centre
[24] 13:23:00.4 Parcel: 529 (134) arr at delivery centre
[24] 13:30:06.9 Parcel: 530 (140) arr at delivery centre
[24] 18:00:00.0 Driver arrives for work
[24] 18:00:00.0 Parcel: 525 (114) out for delivery
[24] 18:00:00.0 Parcel: 523 ( 67) out for delivery
[24] 18:00:00.0 Parcel: 521 ( 41) out for delivery
[24] 18:0

[26] 08:54:52.7 Parcel: 555 (  7) arr at delivery centre
[26] 08:55:51.4 Parcel: 556 ( 14) arr at delivery centre
[26] 08:57:06.9 Parcel: 557 ( 16) arr at delivery centre
[26] 08:57:56.2 Parcel: 558 ( 17) arr at delivery centre
[26] 09:13:03.0 Parcel: 559 ( 18) arr at delivery centre
[26] 09:16:36.0 Parcel: 560 ( 23) arr at delivery centre
[26] 09:29:30.8 Parcel: 561 ( 29) arr at delivery centre
[26] 09:34:41.2 Parcel: 562 ( 32) arr at delivery centre
[26] 09:34:43.7 Parcel: 563 ( 35) arr at delivery centre
[26] 09:47:39.6 Parcel: 564 ( 36) arr at delivery centre
[26] 10:03:46.6 Parcel: 565 ( 38) arr at delivery centre
[26] 10:06:22.6 Parcel: 566 ( 41) arr at delivery centre
[26] 10:12:04.0 Parcel: 567 ( 41) arr at delivery centre
[26] 10:32:47.2 Parcel: 568 ( 45) arr at delivery centre
[26] 10:37:03.3 Parcel: 569 ( 47) arr at delivery centre
[26] 10:55:16.4 Parcel: 570 ( 56) arr at delivery centre
[26] 10:58:37.3 Parcel: 571 ( 65) arr at delivery centre
[26] 11:00:52.8 Parcel: 572 ( 6

[27] 08:48:27.4 Parcel: 590 ( 66) arr at delivery centre
[27] 08:58:20.0 Parcel: 591 ( 68) arr at delivery centre
[27] 09:26:12.7 Parcel: 592 ( 72) arr at delivery centre
[27] 09:52:07.3 Parcel: 593 ( 79) arr at delivery centre
[27] 09:58:34.9 Parcel: 594 ( 86) arr at delivery centre
[27] 10:21:16.8 Parcel: 595 ( 89) arr at delivery centre
[27] 11:14:40.4 Parcel: 596 ( 99) arr at delivery centre
[27] 11:19:31.4 Parcel: 597 (103) arr at delivery centre
[27] 12:09:53.2 Parcel: 598 (107) arr at delivery centre
[27] 12:39:50.5 Parcel: 599 (109) arr at delivery centre
[27] 12:55:20.0 Parcel: 600 (110) arr at delivery centre
[27] 13:17:41.6 Parcel: 601 (122) arr at delivery centre
[27] 13:21:42.5 Parcel: 602 (127) arr at delivery centre
[27] 13:35:42.9 Parcel: 603 (145) arr at delivery centre
[27] 14:03:05.3 Parcel: 604 (148) arr at delivery centre
[27] 14:08:48.2 Parcel: 605 (149) arr at delivery centre
[27] 18:00:00.0 Driver arrives for work
[27] 18:00:00.0 Parcel: 595 ( 89) out for delive

[28] 08:48:03.6 Parcel: 608 ( 23) arr at delivery centre
[28] 09:05:02.0 Parcel: 609 ( 24) arr at delivery centre
[28] 09:24:52.0 Parcel: 610 ( 25) arr at delivery centre
[28] 09:41:22.2 Parcel: 611 ( 49) arr at delivery centre
[28] 10:02:18.8 Parcel: 612 ( 53) arr at delivery centre
[28] 10:47:43.5 Parcel: 613 ( 61) arr at delivery centre
[28] 10:49:09.3 Parcel: 614 ( 62) arr at delivery centre
[28] 10:53:48.2 Parcel: 615 ( 66) arr at delivery centre
[28] 11:00:39.9 Parcel: 616 ( 75) arr at delivery centre
[28] 11:26:26.9 Parcel: 617 ( 77) arr at delivery centre
[28] 11:55:38.2 Parcel: 618 ( 79) arr at delivery centre
[28] 12:14:39.1 Parcel: 619 ( 84) arr at delivery centre
[28] 13:25:43.6 Parcel: 620 (103) arr at delivery centre
[28] 13:43:14.1 Parcel: 621 (111) arr at delivery centre
[28] 13:47:30.9 Parcel: 622 (117) arr at delivery centre
[28] 14:08:19.2 Parcel: 623 (124) arr at delivery centre
[28] 14:29:06.5 Parcel: 624 (135) arr at delivery centre
[28] 14:41:36.5 Parcel: 625 (13

[29] 08:50:22.0 Parcel: 629 ( 25) arr at delivery centre
[29] 09:59:24.2 Parcel: 630 ( 26) arr at delivery centre
[29] 10:02:02.2 Parcel: 631 ( 28) arr at delivery centre
[29] 10:14:01.8 Parcel: 632 ( 39) arr at delivery centre
[29] 10:19:58.9 Parcel: 633 ( 40) arr at delivery centre
[29] 10:23:20.5 Parcel: 634 ( 49) arr at delivery centre
[29] 10:35:51.9 Parcel: 635 ( 75) arr at delivery centre
[29] 10:42:22.9 Parcel: 636 ( 75) arr at delivery centre
[29] 11:30:28.2 Parcel: 637 ( 80) arr at delivery centre
[29] 11:58:10.2 Parcel: 638 ( 86) arr at delivery centre
[29] 12:12:12.1 Parcel: 639 ( 92) arr at delivery centre
[29] 12:29:00.3 Parcel: 640 (115) arr at delivery centre
[29] 12:31:27.9 Parcel: 641 (122) arr at delivery centre
[29] 13:06:57.2 Parcel: 642 (124) arr at delivery centre
[29] 13:57:38.0 Parcel: 643 (135) arr at delivery centre
[29] 14:31:45.5 Parcel: 644 (147) arr at delivery centre
[29] 14:32:39.9 Parcel: 645 (148) arr at delivery centre
[29] 18:00:00.0 Driver arrives 

In [34]:
rec1 = simulation(M, W, C, p=0.15, days=30,seed=6077, log=True, timing=True)
rec1.reportTimer()

generating for day:  0 [3, 8, 18, 20, 27, 32, 34, 40, 45, 50, 52, 55, 69, 71, 73, 74, 79, 85, 86, 91, 94, 101, 122, 124, 135, 135, 145]
generating for day:  1 [1, 8, 10, 13, 34, 39, 46, 58, 67, 71, 78, 81, 100, 100, 108, 127, 127, 129, 134, 137, 140, 141, 143, 147]
generating for day:  2 [0, 3, 6, 15, 18, 25, 32, 34, 35, 42, 42, 47, 49, 52, 53, 61, 74, 77, 91, 96, 102, 114, 119, 121, 132, 142, 146]
generating for day:  3 [10, 14, 17, 29, 29, 31, 41, 58, 72, 85, 89, 96, 109, 110, 112, 124, 141]
generating for day:  4 [12, 19, 20, 51, 58, 75, 76, 86, 92, 98, 100, 107, 135, 137, 140, 146]
generating for day:  5 [8, 11, 12, 29, 30, 76, 78, 82, 84, 110, 117, 119, 125, 126, 126, 134, 145]
generating for day:  6 [2, 8, 14, 15, 25, 40, 55, 90, 97, 103, 112, 116, 120, 141, 149]
generating for day:  7 [4, 10, 15, 20, 30, 33, 37, 48, 61, 67, 68, 82, 82, 96, 98, 101, 101, 104, 108, 108, 116, 118, 127, 133, 136, 141]
generating for day:  8 [10, 14, 15, 17, 33, 34, 46, 50, 51, 56, 63, 77, 91, 92, 94

Solver:           0.23s    656 Constraints
Solver:           0.26s    657 Constraints
Solver:           0.24s    658 Constraints
Solver:           0.29s    659 Constraints
createLoop:       2.37s
==== t:   7.25s Δt:   2.37s [createLoop]
[ 0] 12:47:47.9 Parcel:  24 (135) arr at delivery centre
==== t:   7.25s Δt:   0.01s [addTarget]
createTables:     0.28s
createLoopH:      0.29s
==== t:   7.55s Δt:   0.30s [createLoopH]
createTables:     0.57s
Solver:           0.16s    702 Constraints
Solver:           0.16s    703 Constraints
Solver:           0.17s    704 Constraints
Solver:           0.21s    705 Constraints
Solver:           0.32s    706 Constraints
Solver:           0.25s    707 Constraints
Solver:           0.25s    709 Constraints
Solver:           0.37s    710 Constraints
Solver:           0.31s    712 Constraints
Solver:           0.31s    713 Constraints
Solver:           0.22s    714 Constraints
Solver:           0.40s    715 Constraints
Solver:           0.35s    717 Const

createTables:     0.14s
createLoopH:      0.15s
==== t:  21.57s Δt:   0.15s [createLoopH]
[ 1] 08:19:44.8 Parcel:  29 ( 10) arr at delivery centre
==== t:  21.57s Δt:   0.00s [addTarget]
createTables:     0.14s
createLoopH:      0.14s
==== t:  21.70s Δt:   0.14s [createLoopH]
[ 1] 08:43:16.5 Parcel:  30 ( 13) arr at delivery centre
==== t:  21.71s Δt:   0.00s [addTarget]
createTables:     0.15s
createLoopH:      0.15s
==== t:  21.86s Δt:   0.15s [createLoopH]
[ 1] 08:45:44.6 Parcel:  31 ( 34) arr at delivery centre
==== t:  21.86s Δt:   0.00s [addTarget]
createTables:     0.14s
createLoopH:      0.14s
==== t:  22.00s Δt:   0.14s [createLoopH]
[ 1] 09:15:03.3 Parcel:  32 ( 39) arr at delivery centre
==== t:  22.00s Δt:   0.00s [addTarget]
createTables:     0.15s
createLoopH:      0.15s
==== t:  22.15s Δt:   0.15s [createLoopH]
[ 1] 09:35:09.7 Parcel:  33 ( 46) arr at delivery centre
==== t:  22.15s Δt:   0.00s [addTarget]
createTables:     0.25s
createLoopH:      0.25s
==== t:  22.41s Δ

createTables:     0.13s
createLoopH:      0.13s
==== t:  32.98s Δt:   0.13s [createLoopH]
[ 2] 08:42:14.8 Parcel:  52 (  3) arr at delivery centre
==== t:  32.99s Δt:   0.00s [addTarget]
createTables:     0.13s
createLoopH:      0.13s
==== t:  33.12s Δt:   0.13s [createLoopH]
[ 2] 08:55:40.6 Parcel:  53 (  6) arr at delivery centre
==== t:  33.12s Δt:   0.00s [addTarget]
createTables:     0.13s
createLoopH:      0.13s
==== t:  33.25s Δt:   0.13s [createLoopH]
[ 2] 09:01:35.9 Parcel:  54 ( 15) arr at delivery centre
==== t:  33.25s Δt:   0.01s [addTarget]
createTables:     0.14s
createLoopH:      0.14s
==== t:  33.39s Δt:   0.14s [createLoopH]
[ 2] 09:21:40.9 Parcel:  55 ( 18) arr at delivery centre
==== t:  33.39s Δt:   0.00s [addTarget]
createTables:     0.22s
createLoopH:      0.22s
==== t:  33.61s Δt:   0.22s [createLoopH]
[ 2] 09:27:47.4 Parcel:  56 ( 25) arr at delivery centre
==== t:  33.62s Δt:   0.00s [addTarget]
createTables:     0.16s
createLoopH:      0.16s
==== t:  33.77s Δ

createTables:     0.13s
createLoopH:      0.13s
==== t:  45.22s Δt:   0.14s [createLoopH]
==== t:  45.22s Δt:   0.00s [addTarget]
createTables:     0.14s
createLoopH:      0.14s
==== t:  45.36s Δt:   0.14s [createLoopH]
[ 2] 18:20:00.0 Driver leaves for delivery of 24 parcels
[ 2] 18:20:00.0 Driver drives to Customer: 114 (5120, 6625)
[ 2] 18:20:42.0 Driver arrived at Customer: 114 (5120, 6625)
[ 2] 18:20:57.8 Customer: 114 (5120, 6625) answers door
[ 2] 18:20:57.8 Customer: 114 (5120, 6625) accepts Parcel:  72 (114)
[ 2] 18:21:06.7 Customer: 114 (5120, 6625) signs off
[ 2] 18:21:07.6 Driver drives to Customer: 91 (4359, 4560)
[ 2] 18:32:25.9 Driver arrived at Customer: 91 (4359, 4560)
[ 2] 18:35:29.6 Customer: 91 (4359, 4560) answers door
[ 2] 18:35:29.6 Customer: 91 (4359, 4560) accepts Parcel:  69 ( 91)
[ 2] 18:35:36.9 Customer: 91 (4359, 4560) signs off
[ 2] 18:35:47.1 Driver drives to Customer: 77 (4000, 4356)
[ 2] 18:38:02.2 Driver arrived at Customer: 77 (4000, 4356)
[ 2] 18:38:

...

createTables:     0.14s
createLoopH:      0.14s
==== t: 921.32s Δt:   0.14s [createLoopH]
[26] 08:54:52.7 Parcel: 555 (  7) arr at delivery centre
==== t: 921.32s Δt:   0.00s [addTarget]
createTables:     0.13s
createLoopH:      0.13s
==== t: 921.45s Δt:   0.13s [createLoopH]
[26] 08:55:51.4 Parcel: 556 ( 14) arr at delivery centre
==== t: 921.45s Δt:   0.00s [addTarget]
createTables:     0.15s
createLoopH:      0.16s
==== t: 921.60s Δt:   0.16s [createLoopH]
[26] 08:57:06.9 Parcel: 557 ( 16) arr at delivery centre
==== t: 921.61s Δt:   0.00s [addTarget]
createTables:     0.16s
createLoopH:      0.16s
==== t: 921.76s Δt:   0.16s [createLoopH]
[26] 08:57:56.2 Parcel: 558 ( 17) arr at delivery centre
==== t: 921.76s Δt:   0.00s [addTarget]
createTables:     0.16s
createLoopH:      0.16s
==== t: 921.93s Δt:   0.17s [createLoopH]
[26] 09:13:03.0 Parcel: 559 ( 18) arr at delivery centre
==== t: 921.93s Δt:   0.00s [addTarget]
createTables:     0.22s
createLoopH:      0.22s
==== t: 922.16s Δ

createTables:     0.33s
createLoopH:      0.37s
==== t: 947.72s Δt:   0.37s [createLoopH]
createTables:     0.72s
Solver:           0.24s    992 Constraints
Solver:           0.23s    994 Constraints
Solver:           0.23s    995 Constraints
Solver:           0.18s    996 Constraints
Solver:           0.23s    997 Constraints
Solver:           0.26s    998 Constraints
Solver:           0.36s    999 Constraints
Solver:           0.31s  1,000 Constraints
Solver:           0.36s  1,001 Constraints
Solver:           0.44s  1,003 Constraints
Solver:           0.28s  1,004 Constraints
Solver:           0.37s  1,005 Constraints
Solver:           0.82s  1,006 Constraints
createLoop:       5.03s
==== t: 952.75s Δt:   5.03s [createLoop]
[26] 13:54:41.6 Parcel: 585 (142) arr at delivery centre
==== t: 952.75s Δt:   0.01s [addTarget]
createTables:     0.37s
createLoopH:      0.41s
==== t: 953.16s Δt:   0.41s [createLoopH]
createTables:     0.75s
Solver:           0.20s  1,056 Constraints
Solver: 

createTables:     0.23s
createLoopH:      0.23s
==== t: 978.57s Δt:   0.23s [createLoopH]
[27] 08:48:27.4 Parcel: 590 ( 66) arr at delivery centre
==== t: 978.57s Δt:   0.00s [addTarget]
createTables:     0.15s
createLoopH:      0.15s
==== t: 978.72s Δt:   0.15s [createLoopH]
[27] 08:58:20.0 Parcel: 591 ( 68) arr at delivery centre
==== t: 978.72s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.18s
==== t: 978.90s Δt:   0.18s [createLoopH]
[27] 09:26:12.7 Parcel: 592 ( 72) arr at delivery centre
==== t: 978.90s Δt:   0.00s [addTarget]
createTables:     0.16s
createLoopH:      0.16s
==== t: 979.06s Δt:   0.16s [createLoopH]
[27] 09:52:07.3 Parcel: 593 ( 79) arr at delivery centre
==== t: 979.07s Δt:   0.00s [addTarget]
createTables:     0.16s
createLoopH:      0.16s
==== t: 979.22s Δt:   0.16s [createLoopH]
[27] 09:58:34.9 Parcel: 594 ( 86) arr at delivery centre
==== t: 979.22s Δt:   0.00s [addTarget]
createTables:     0.17s
createLoopH:      0.17s
==== t: 979.40s Δ

createTables:     0.14s
createLoopH:      0.14s
==== t: 984.00s Δt:   0.14s [createLoopH]
[28] 08:48:03.6 Parcel: 608 ( 23) arr at delivery centre
==== t: 984.00s Δt:   0.00s [addTarget]
createTables:     0.17s
createLoopH:      0.17s
==== t: 984.17s Δt:   0.17s [createLoopH]
[28] 09:05:02.0 Parcel: 609 ( 24) arr at delivery centre
==== t: 984.17s Δt:   0.00s [addTarget]
createTables:     0.14s
createLoopH:      0.14s
==== t: 984.32s Δt:   0.15s [createLoopH]
[28] 09:24:52.0 Parcel: 610 ( 25) arr at delivery centre
==== t: 984.32s Δt:   0.00s [addTarget]
createTables:     0.16s
createLoopH:      0.16s
==== t: 984.48s Δt:   0.16s [createLoopH]
[28] 09:41:22.2 Parcel: 611 ( 49) arr at delivery centre
==== t: 984.48s Δt:   0.00s [addTarget]
createTables:     0.16s
createLoopH:      0.16s
==== t: 984.64s Δt:   0.16s [createLoopH]
[28] 10:02:18.8 Parcel: 612 ( 53) arr at delivery centre
==== t: 984.64s Δt:   0.00s [addTarget]
createTables:     0.18s
createLoopH:      0.18s
==== t: 984.82s Δ

createTables:     0.14s
createLoopH:      0.14s
==== t: 991.06s Δt:   0.14s [createLoopH]
[29] 08:50:22.0 Parcel: 629 ( 25) arr at delivery centre
==== t: 991.06s Δt:   0.00s [addTarget]
createTables:     0.22s
createLoopH:      0.22s
==== t: 991.28s Δt:   0.22s [createLoopH]
[29] 09:59:24.2 Parcel: 630 ( 26) arr at delivery centre
==== t: 991.28s Δt:   0.00s [addTarget]
createTables:     0.15s
createLoopH:      0.15s
==== t: 991.43s Δt:   0.15s [createLoopH]
[29] 10:02:02.2 Parcel: 631 ( 28) arr at delivery centre
==== t: 991.43s Δt:   0.00s [addTarget]
createTables:     0.17s
createLoopH:      0.17s
==== t: 991.60s Δt:   0.17s [createLoopH]
[29] 10:14:01.8 Parcel: 632 ( 39) arr at delivery centre
==== t: 991.60s Δt:   0.00s [addTarget]
createTables:     0.15s
createLoopH:      0.15s
==== t: 991.75s Δt:   0.15s [createLoopH]
[29] 10:19:58.9 Parcel: 633 ( 40) arr at delivery centre
==== t: 991.75s Δt:   0.00s [addTarget]
createTables:     0.15s
createLoopH:      0.15s
==== t: 991.90s Δ